In [27]:
import pandas as pd
import numpy as np

In [10]:
reviews_tsv = pd.read_csv('reviews.tsv', sep='\t', header=0, encoding='unicode_escape')
reviews_tsv.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [46]:
a = reviews_tsv['review'].values
a = np.array2string(a)
a = a.replace('"', '').replace("'", '').replace('\n','').replace(',','').replace('[','').replace(']','')
a.lower().split()

['a',
 'distinctly',
 'gallows',
 'take',
 'on',
 'contemporary',
 'financial',
 'mores',
 'as',
 'one',
 'absurdly',
 'rich',
 'mans',
 'limo',
 'ride',
 'across',
 'town',
 'for',
 'a',
 'haircut',
 'functions',
 'as',
 'a',
 'state-of-the-nation',
 'discourse.',
 'its',
 'an',
 'allegory',
 'in',
 'search',
 'of',
 'a',
 'meaning',
 'that',
 'never',
 'arrives...its',
 'just',
 'old-fashioned',
 'bad',
 'storytelling.',
 '...',
 'life',
 'lived',
 'in',
 'a',
 'bubble',
 'in',
 'financial',
 'dealings',
 'and',
 'digital',
 'communications',
 'and',
 'brief',
 'face-to-face',
 'conversations',
 'and',
 'sexual',
 'intermissions',
 'in',
 'a',
 'space',
 'shuttle',
 'of',
 'a',
 'limousine',
 'creeping',
 'through',
 'the',
 'gridlock',
 'of',
 'an',
 'anonymous',
 'new',
 'york',
 'city.',
 '...',
 'nan',
 'nan',
 'nan']